In [1]:
!pip install streamlit transformers pyngrok --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 46.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 52.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 4.9 MB/s eta 0:00:00


In [2]:
!ngrok config add-authtoken 2wrQddqdmJVbR8EMEMDZA6ZD7FH_7rx9RwkLi57ygGGUNouqB

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [4]:
%%writefile app.py
import streamlit as st
from transformers import pipeline, AutoTokenizer

# Load tokenizer to count tokens properly
tokenizer = AutoTokenizer.from_pretrained("google/pegasus-xsum")

# Set page config
st.set_page_config(page_title="📰 News Summarizer", layout="wide")

# Inject custom CSS for styling
st.markdown("""
<style>
    .headline-box {
        background: linear-gradient(90deg, #4b6cb7 0%, #182848 100%);
        padding: 20px;
        border-radius: 10px;
        color: white;
        font-size: 22px;
        font-weight: bold;
        margin-bottom: 20px;
    }
    .bullet-point {
        background-color: #f1f3f6;
        color: #000000;
        padding: 10px;
        border-radius: 8px;
        margin-bottom: 8px;
        font-size: 16px;
    }
    .summary-box {
        background-color: #e3f2fd;
        padding: 15px;
        border-radius: 8px;
        color: #333;
        font-size: 17px;
    }
    .section-header {
        font-size: 24px;
        margin-top: 30px;
        color: ##0000FF;
        font-weight: 700;
    }
</style>
""", unsafe_allow_html=True)

# Load NLP models once
@st.cache_resource
def load_pipelines():
    summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
    headline_gen = pipeline("summarization", model="google/pegasus-xsum")
    return summarizer, headline_gen

summarizer, headline_gen = load_pipelines()

# Summarization logic
def summarize_article(text):
    chunks = [text[i:i+1000] for i in range(0, len(text), 1000)]
    summary = ""
    for chunk in chunks:
        out = summarizer(chunk, max_length=130, min_length=30, do_sample=False)
        summary += out[0]['summary_text'] + " "
    return summary.strip()

def generate_headline(text, max_input_tokens=512):
    text = text.strip()
    if not text:
        return "No headline generated (empty article)."

    # Tokenize and truncate the input
    inputs = tokenizer.encode(text, truncation=True, max_length=max_input_tokens, return_tensors="pt")
    decoded_input = tokenizer.decode(inputs[0], skip_special_tokens=True)

    # Generate headline
    result = headline_gen(decoded_input, max_length=15, min_length=5, do_sample=False)

    return result[0]['summary_text']

def extract_subpoints(summary):
    sentences = summary.split('. ')
    return [s.strip() for s in sentences if s][:5]

# Sidebar info
with st.sidebar:
    st.image("https://img.icons8.com/color/96/news.png", width=80)
    st.markdown("## 📰 News Summarizer")
    st.markdown("""
    This app takes a news article and generates:
    - 🏷️ A headline
    - 📌 Bullet points
    - 📝 A summary paragraph
    """)
    st.markdown("---")
    st.caption("Made by Gurleen Kaur Makhija")

# App title
st.markdown("<h1 style='text-align: center; color: #4b6cb7;'>✨ News Summarizer & Headline Generator</h1>", unsafe_allow_html=True)
st.markdown("---")

# Text input
article = st.text_area("✍️ Paste your news article below:", height=300)
st.caption(f"📝 Word Count: {len(article.split())}")

# Generate button
if st.button("🚀 Generate Summary"):
    if len(article.split()) < 30:
        st.warning("Please enter at least 30 words.")
    else:
        with st.spinner("Analyzing and summarizing..."):
            headline = generate_headline(article)
            summary = summarize_article(article)
            subpoints = extract_subpoints(summary)

        # Headline
        st.markdown("<div class='section-header'>🗞️ Generated Headline</div>", unsafe_allow_html=True)
        st.markdown(f"<div class='headline-box'>{headline}</div>", unsafe_allow_html=True)

        # Bullet Points
        st.markdown("<div class='section-header'>📌 Key Points</div>", unsafe_allow_html=True)
        for pt in subpoints:
            st.markdown(f"<div class='bullet-point'>🔹 {pt}</div>", unsafe_allow_html=True)

        # Summary
        st.markdown("<div class='section-header'>📝 Summary Paragraph</div>", unsafe_allow_html=True)
        st.markdown(f"<div class='summary-box'>{summary}</div>", unsafe_allow_html=True)


Overwriting app.py


In [5]:
from pyngrok import ngrok

ngrok.kill()
!streamlit run app.py &> /content/log.txt &
public_url = ngrok.connect(8501, "http")
print("🌐 Open your app here:", public_url)

🌐 Open your app here: NgrokTunnel: "https://1f45-34-125-254-108.ngrok-free.app" -> "http://localhost:8501"
